In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from data_utility import DeepiqDataset
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset, DataLoader, random_split, Dataset
import torch

In [19]:
BATCH_SIZE = 32

In [20]:
full_dataset = DeepiqDataset(
    dataset_root_dir = "./odd-one-out",
    num_panels = [5]
)

dataset_size = len(full_dataset)
print(f"Dataset size: {dataset_size}")

train_prop = 0.6
val_prop = 0.2
# test_prop = 0.2

train_size = int(train_prop * dataset_size)
val_size = int(val_prop * dataset_size)

test_size = dataset_size - train_size - val_size

print(f"Splitting into:")
print(f"  - Training set:   {train_size} samples ({train_size/dataset_size:.2%})")
print(f"  - Validation set: {val_size} samples ({val_size/dataset_size:.2%})")
print(f"  - Test set:       {test_size} samples ({test_size/dataset_size:.2%})")
print("-" * 40)


generator = torch.Generator().manual_seed(42)

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size], generator=generator
)


# --- 4. Verification ---
print(f"Type of train_dataset: {type(train_dataset)}")

# Verify the lengths of the splits
print(f"Actual size of train_dataset: {len(train_dataset)}")
print(f"Actual size of val_dataset:   {len(val_dataset)}")
print(f"Actual size of test_dataset:  {len(test_dataset)}")

assert len(train_dataset) + len(val_dataset) + len(test_dataset) == dataset_size
print("\nSuccessfully split the dataset without losing any data points.")
print("-" * 40)


# --- 5. Create DataLoaders for each set ---
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Number of batches in train_loader: {len(train_loader)}")
print(f"Number of batches in val_loader:   {len(val_loader)}")
print(f"Number of batches in test_loader:  {len(test_loader)}")

Dataset size: 500
Splitting into:
  - Training set:   300 samples (60.00%)
  - Validation set: 100 samples (20.00%)
  - Test set:       100 samples (20.00%)
----------------------------------------
Type of train_dataset: <class 'torch.utils.data.dataset.Subset'>
Actual size of train_dataset: 300
Actual size of val_dataset:   100
Actual size of test_dataset:  100

Successfully split the dataset without losing any data points.
----------------------------------------
Number of batches in train_loader: 10
Number of batches in val_loader:   4
Number of batches in test_loader:  4


In [21]:
from msrgnn import OddOneOutMSRGNN, validate, train_epoch, count_parameters

In [22]:
import torch
import torch.nn as nn
import time

In [23]:
CTX_SIZE = 4
CAND_SIZE = 4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LEARNING_RATE = 1e-4
EPOCHS = 100

model_params = {
    'resnet_base_channels': 64,
    'pos_embedding_dim': 128,
    'gnn_hidden_dim': 128,
    'relation_net_mlp_layers': 3,
    'relation_net_dropout': 0,
    'gnn_message_dropout_p': 0,
    'reasoner_proj_dropout': 0,
    'num_panels': 5,
}

model = OddOneOutMSRGNN(**model_params).to(DEVICE)

In [24]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
scaler = None
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=EPOCHS,
    eta_min=1e-6
)

In [25]:
import json

In [26]:
# --- Training Hyperparameters ---
N_SPLITS = 3
EPOCHS = 100
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-4
PATIENCE = 10 # For early stopping

# --- Model & Logging Setup ---
MODEL_FOLDER_NAME = "O3_5-MSRGNN"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [27]:
import os
import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

# O3 STL

In [28]:
from sklearn.model_selection import train_test_split
VAL_SPLIT_SIZE = 0.15

# ==============================================================================
# 2. DATASET & FOLD SETUP
# ==============================================================================
print(f"Total data for cross-validation: {len(full_dataset)} samples.")

kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
fold_test_results = []

# ==============================================================================
# 3. CROSS-VALIDATION LOOP
# ==============================================================================
for fold, (train_outer_idx, test_outer_idx) in enumerate(kfold.split(np.zeros(len(full_dataset)))):
    print(f"\n{'='*30} OUTER FOLD {fold+1}/{N_SPLITS} {'='*30}")

    SAVED_MODELS_PATH = f"./saved_models/{MODEL_FOLDER_NAME}/"

    if not os.path.exists(SAVED_MODELS_PATH):
        os.makedirs(SAVED_MODELS_PATH)
    writer = SummaryWriter(log_dir=f'runs/{MODEL_FOLDER_NAME}/fold_{fold+1}')

    inner_train_idx, inner_val_idx = train_test_split(
        train_outer_idx,
        test_size=VAL_SPLIT_SIZE,
        shuffle=True,
        random_state=42
    )

    inner_train_subset = Subset(full_dataset, inner_train_idx)
    inner_val_subset = Subset(full_dataset, inner_val_idx)
    outer_test_subset = Subset(full_dataset, test_outer_idx)

    train_loader = DataLoader(inner_train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(inner_val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(outer_test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    model = OddOneOutMSRGNN(**model_params).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    criterion = nn.CrossEntropyLoss()
    scaler = None
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Fold {fold+1} Model has {total_params:,} trainable parameters.")
    print(f"Fold {fold+1} Training on {len(inner_train_subset)}, Validating on {len(inner_val_subset)}, Testing on {len(outer_test_subset)}.")

    best_model_path = f"{SAVED_MODELS_PATH}/best_model_fold_{fold+1}.pth"
    best_val_acc_fold = 0.0
    epochs_without_improvement = 0

    for epoch in range(1, EPOCHS + 1):
        epoch_start_time = time.time()
        
        print(f"\n--- Fold {fold+1}, Epoch {epoch}/{EPOCHS} ---")
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, DEVICE)
        val_loss, val_acc = validate(model, val_loader, criterion, DEVICE)

        scheduler.step()

        epoch_duration = time.time() - epoch_start_time

        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f}")
        print(f"  Epoch Duration: {epoch_duration:.2f} seconds")
        
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('Accuracy/val', val_acc, epoch)
        writer.add_scalar('Performance/seconds_per_epoch', epoch_duration, epoch)
        writer.add_scalar('Learning_Rate', optimizer.param_groups[0]['lr'], epoch)

        if val_acc > best_val_acc_fold:
            best_val_acc_fold = val_acc
            epochs_without_improvement = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"  -> New best inner validation accuracy: {best_val_acc_fold:.4f}. Model state saved.")
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= PATIENCE:
                print(f"  Early stopping at epoch {epoch}.")
                break
    
    print(f"\n--- Fold {fold+1} Final Evaluation ---")
    print(f"Loading best model for fold {fold+1} (achieved {best_val_acc_fold:.4f} on inner val set).")
    # Load the best model weights identified during the inner loop
    model.load_state_dict(torch.load(best_model_path))
    
    # Evaluate this best model on the completely unseen outer test set
    _, final_fold_test_acc = validate(model, test_loader, criterion, DEVICE)
    print(f"Performance of best model on the OUTER TEST SET for fold {fold+1}: {final_fold_test_acc:.4f}")

    fold_test_results.append(final_fold_test_acc)
    writer.add_scalar('Accuracy/test', final_fold_test_acc)

    hparams = { 'learning_rate': LEARNING_RATE, 'batch_size': BATCH_SIZE, **model_params }
    final_metrics = {
        'hparam/test_accuracy': final_fold_test_acc,
        'hparam/best_inner_val_accuracy': best_val_acc_fold
    }
    writer.add_hparams(hparams, final_metrics)
    writer.close()

# ==============================================================================
# 4. AGGREGATE AND REPORT CV RESULTS
# ==============================================================================
mean_test_acc = np.mean(fold_test_results)
std_test_acc = np.std(fold_test_results)

print(f"\n{'='*30} FINAL UNBIASED PERFORMANCE SUMMARY {'='*30}")
print(f"Unbiased test accuracies from each fold: {[f'{acc:.4f}' for acc in fold_test_results]}")
print(f"Average Test Accuracy from CV: {mean_test_acc:.4f}")
print(f"Standard Deviation of Test Accuracy from CV: {std_test_acc:.4f}")
print(f"-> Estimated Generalization Performance: {mean_test_acc:.4f} \u00B1 {std_test_acc:.4f}")

print("\nCross-validation finished.")
print("To view TensorBoard logs, run the following command in your terminal:")
print(f"tensorboard --logdir runs/{MODEL_FOLDER_NAME}")

Total data for cross-validation: 500 samples.

============================== OUTER FOLD 1/3 ==============================
Fold 1 Model has 4,804,547 trainable parameters.
Fold 1 Training on 283, Validating on 50, Testing on 167.

--- Fold 1, Epoch 1/100 ---


Training:  11%|█         | 1/9 [00:00<00:03,  2.13it/s]

Training: 100%|██████████| 9/9 [00:01<00:00,  8.22it/s]


Epoch Loss: 1.5386, Accuracy: 0.3993


Validating: 100%|██████████| 2/2 [00:00<00:00,  4.12it/s]


...

Epoch Loss: 0.0112, Accuracy: 1.0000


Validating: 100%|██████████| 2/2 [00:00<00:00,  4.09it/s]


  Train Loss: 0.0112 | Train Acc: 1.0000
  Val Loss:   2.5753 | Val Acc:   0.2941
  Epoch Duration: 1.72 seconds
  Early stopping at epoch 15.

--- Fold 3 Final Evaluation ---
Loading best model for fold 3 (achieved 0.3725 on inner val set).


Validating: 100%|██████████| 6/6 [00:00<00:00,  9.17it/s]

Performance of best model on the OUTER TEST SET for fold 3: 0.2831

============================== FINAL UNBIASED PERFORMANCE SUMMARY ==============================
Unbiased test accuracies from each fold: ['0.3952', '0.3413', '0.2831']
Average Test Accuracy from CV: 0.3399
Standard Deviation of Test Accuracy from CV: 0.0458
-> Estimated Generalization Performance: 0.3399 ± 0.0458

Cross-validation finished.
To view TensorBoard logs, run the following command in your terminal:
tensorboard --logdir runs/O3_5-MSRGNN


In [29]:
def load_partial_state_dict(model_to_load: nn.Module, pretrained_path: str):
    """
    Loads weights from a pretrained model, skipping layers with mismatched shapes.

    Args:
        model_to_load (nn.Module): The model instance to load weights INTO.
        pretrained_path (str): The path to the .pt or .pth file of the pretrained model.
    """
    print(f"Loading pretrained weights from: {pretrained_path}")
    
    pretrained_dict = torch.load(pretrained_path, map_location=torch.device('cpu'))

    if 'model_state_dict' in pretrained_dict:
        pretrained_dict = pretrained_dict['model_state_dict']
    elif 'model' in pretrained_dict and isinstance(pretrained_dict['model'], dict):
        pretrained_dict = pretrained_dict['model']
        
    model_dict = model_to_load.state_dict()

    # 1. Filter out unnecessary keys and weights with mismatched shapes
    # We create a new dictionary that only contains the weights we can actually use
    filtered_dict = {}
    initialized_layers = []
    skipped_layers = []

    for key, param in pretrained_dict.items():
        if key in model_dict:
            # Check if the shape of the weights match
            if param.shape == model_dict[key].shape:
                filtered_dict[key] = param
                initialized_layers.append(key)
            else:
                # Key exists, but shapes are different. This is the main case we want to handle.
                reason = f"Shape Mismatch. Pretrained: {param.shape}, Model: {model_dict[key].shape}"
                skipped_layers.append((key, reason))
        else:
            # The key from the pretrained model doesn't exist in the new model
            skipped_layers.append((key, "Key Not Found in Model"))

    # 2. Overwrite the entries in the model's state dict with the filtered weights
    model_dict.update(filtered_dict)

    # 3. Load the updated state dict into the model.
    # Since we've filtered it, we can be strict here.
    model_to_load.load_state_dict(model_dict)

    print("\n--- Transfer Learning Report ---")
    print(f"Successfully loaded {len(initialized_layers)} / {len(model_dict)} layers.")
    if initialized_layers and len(initialized_layers) < 10:
        print("Loaded layers:")
        for layer in initialized_layers:
            print(f"  - {layer}")
    
    if skipped_layers:
        print(f"\nSkipped {len(skipped_layers)} layers:")
        for key, reason in skipped_layers:
            print(f"  - {key} ({reason})")
    print("--------------------------------\n")
    
    return model_to_load

# I-RAVEN to O3 TL

In [30]:
from sklearn.model_selection import train_test_split
VAL_SPLIT_SIZE = 0.15

MODEL_FOLDER_NAME = "O3_5-MSRGNN-TRANSFER"

# ==============================================================================
# 2. DATASET & FOLD SETUP
# ==============================================================================
print(f"Total data for cross-validation: {len(full_dataset)} samples.")

kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
fold_test_results = []

# ==============================================================================
# 3. CROSS-VALIDATION LOOP
# ==============================================================================
for fold, (train_outer_idx, test_outer_idx) in enumerate(kfold.split(np.zeros(len(full_dataset)))):
    print(f"\n{'='*30} OUTER FOLD {fold+1}/{N_SPLITS} {'='*30}")

    SAVED_MODELS_PATH = f"./saved_models/{MODEL_FOLDER_NAME}/"

    if not os.path.exists(SAVED_MODELS_PATH):
        os.makedirs(SAVED_MODELS_PATH)
    writer = SummaryWriter(log_dir=f'runs/{MODEL_FOLDER_NAME}/fold_{fold+1}')

    inner_train_idx, inner_val_idx = train_test_split(
        train_outer_idx,
        test_size=VAL_SPLIT_SIZE,
        shuffle=True, # Shuffle before splitting
        random_state=42
    )

    inner_train_subset = Subset(full_dataset, inner_train_idx)
    inner_val_subset = Subset(full_dataset, inner_val_idx)
    outer_test_subset = Subset(full_dataset, test_outer_idx)

    train_loader = DataLoader(inner_train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(inner_val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(outer_test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    model = OddOneOutMSRGNN(**model_params).to(DEVICE)

    load_partial_state_dict(
        model,
        "../I_RAVEN/saved_models/best_model_msrgnn.pth"
    )

    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
    scaler = None

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Fold {fold+1} Model has {total_params:,} trainable parameters.")
    print(f"Fold {fold+1} Training on {len(inner_train_subset)}, Validating on {len(inner_val_subset)}, Testing on {len(outer_test_subset)}.")

    best_model_path = f"{SAVED_MODELS_PATH}/best_model_fold_{fold+1}.pth"
    best_val_acc_fold = 0.0
    epochs_without_improvement = 0

    for epoch in range(1, EPOCHS + 1):
        epoch_start_time = time.time()
        
        print(f"\n--- Fold {fold+1}, Epoch {epoch}/{EPOCHS} ---")
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, DEVICE)
        val_loss, val_acc = validate(model, val_loader, criterion, DEVICE)

        scheduler.step()

        epoch_duration = time.time() - epoch_start_time

        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f}")
        print(f"  Epoch Duration: {epoch_duration:.2f} seconds")
        
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('Accuracy/val', val_acc, epoch)
        writer.add_scalar('Performance/seconds_per_epoch', epoch_duration, epoch)
        writer.add_scalar('Learning_Rate', optimizer.param_groups[0]['lr'], epoch)

        if val_acc > best_val_acc_fold:
            best_val_acc_fold = val_acc
            epochs_without_improvement = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"  -> New best inner validation accuracy: {best_val_acc_fold:.4f}. Model state saved.")
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= PATIENCE:
                print(f"  Early stopping at epoch {epoch}.")
                break
    
    print(f"\n--- Fold {fold+1} Final Evaluation ---")
    print(f"Loading best model for fold {fold+1} (achieved {best_val_acc_fold:.4f} on inner val set).")
    # Load the best model weights identified during the inner loop
    model.load_state_dict(torch.load(best_model_path))
    
    # Evaluate this best model on the completely unseen outer test set
    _, final_fold_test_acc = validate(model, test_loader, criterion, DEVICE)
    print(f"Performance of best model on the OUTER TEST SET for fold {fold+1}: {final_fold_test_acc:.4f}")

    fold_test_results.append(final_fold_test_acc)
    writer.add_scalar('Accuracy/test', final_fold_test_acc)
    
    hparams = { 'learning_rate': LEARNING_RATE, 'batch_size': BATCH_SIZE, **model_params }
    final_metrics = {
        'hparam/test_accuracy': final_fold_test_acc,
        'hparam/best_inner_val_accuracy': best_val_acc_fold
    }
    writer.add_hparams(hparams, final_metrics)
    writer.close()

# ==============================================================================
# 4. AGGREGATE AND REPORT CV RESULTS
# ==============================================================================
mean_test_acc = np.mean(fold_test_results)
std_test_acc = np.std(fold_test_results)

print(f"\n{'='*30} FINAL UNBIASED PERFORMANCE SUMMARY {'='*30}")
print(f"Unbiased test accuracies from each fold: {[f'{acc:.4f}' for acc in fold_test_results]}")
print(f"Average Test Accuracy from CV: {mean_test_acc:.4f}")
print(f"Standard Deviation of Test Accuracy from CV: {std_test_acc:.4f}")
print(f"-> Estimated Generalization Performance: {mean_test_acc:.4f} \u00B1 {std_test_acc:.4f}")

print("\nCross-validation finished.")
print("To view TensorBoard logs, run the following command in your terminal:")
print(f"tensorboard --logdir runs/{MODEL_FOLDER_NAME}")

Total data for cross-validation: 500 samples.

============================== OUTER FOLD 1/3 ==============================
Loading pretrained weights from: ../I_RAVEN/saved_models/best_model_msrgnn.pth

--- Transfer Learning Report ---
Successfully loaded 138 / 140 layers.

Skipped 2 layers:
  - template_edge_index (Shape Mismatch. Pretrained: torch.Size([2, 36]), Model: torch.Size([2, 12]))
  - reasoner.panel_pos_emb.weight (Shape Mismatch. Pretrained: torch.Size([9, 128]), Model: torch.Size([4, 128]))
--------------------------------

Fold 1 Model has 4,804,547 trainable parameters.
Fold 1 Training on 283, Validating on 50, Testing on 167.

--- Fold 1, Epoch 1/100 ---


Training: 100%|██████████| 9/9 [00:01<00:00,  7.98it/s]


Epoch Loss: 1.4550, Accuracy: 0.4841


Validating: 100%|██████████| 2/2 [00:00<00:00,  4.28it/s]


  Train Loss: 1.4550 | Train Acc: 0.4841
  Val Loss:   1.6072 | Val Acc:   0.3200
  Epoch Duration: 1.60 seconds
  -> New best inner validation accuracy: 0.3200. Model state saved.

--- Fold 1, Epoch 2/100 ---


...

Epoch Loss: 0.0001, Accuracy: 1.0000


Validating: 100%|██████████| 2/2 [00:00<00:00,  3.91it/s]


  Train Loss: 0.0001 | Train Acc: 1.0000
  Val Loss:   1.8771 | Val Acc:   0.6275
  Epoch Duration: 1.81 seconds
  Early stopping at epoch 20.

--- Fold 3 Final Evaluation ---
Loading best model for fold 3 (achieved 0.6471 on inner val set).


Validating: 100%|██████████| 6/6 [00:00<00:00,  9.65it/s]

Performance of best model on the OUTER TEST SET for fold 3: 0.6325

============================== FINAL UNBIASED PERFORMANCE SUMMARY ==============================
Unbiased test accuracies from each fold: ['0.6946', '0.7365', '0.6325']
Average Test Accuracy from CV: 0.6879
Standard Deviation of Test Accuracy from CV: 0.0427
-> Estimated Generalization Performance: 0.6879 ± 0.0427

Cross-validation finished.
To view TensorBoard logs, run the following command in your terminal:
tensorboard --logdir runs/O3_5-MSRGNN-TRANSFER


# RADIO-1 to O3 TL

In [31]:
from sklearn.model_selection import train_test_split
VAL_SPLIT_SIZE = 0.15

MODEL_FOLDER_NAME = "O3_5-MSRGNN-TRANSFER_RADIO"

# ==============================================================================
# 2. DATASET & FOLD SETUP
# ==============================================================================
print(f"Total data for cross-validation: {len(full_dataset)} samples.")

kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
fold_test_results = []

# ==============================================================================
# 3. CROSS-VALIDATION LOOP
# ==============================================================================
for fold, (train_outer_idx, test_outer_idx) in enumerate(kfold.split(np.zeros(len(full_dataset)))):
    print(f"\n{'='*30} OUTER FOLD {fold+1}/{N_SPLITS} {'='*30}")

    SAVED_MODELS_PATH = f"./saved_models/{MODEL_FOLDER_NAME}/"

    if not os.path.exists(SAVED_MODELS_PATH):
        os.makedirs(SAVED_MODELS_PATH)
    writer = SummaryWriter(log_dir=f'runs/{MODEL_FOLDER_NAME}/fold_{fold+1}')

    inner_train_idx, inner_val_idx = train_test_split(
        train_outer_idx,
        test_size=VAL_SPLIT_SIZE,
        shuffle=True, # Shuffle before splitting
        random_state=42
    )

    inner_train_subset = Subset(full_dataset, inner_train_idx)
    inner_val_subset = Subset(full_dataset, inner_val_idx)
    outer_test_subset = Subset(full_dataset, test_outer_idx)

    train_loader = DataLoader(inner_train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(inner_val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(outer_test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    model = OddOneOutMSRGNN(**model_params).to(DEVICE)

    load_partial_state_dict(
        model,
        "../RADIO/saved_models/RADIO1_MSRGNN/deployable_model.pth"
    )

    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
    scaler = None

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Fold {fold+1} Model has {total_params:,} trainable parameters.")
    print(f"Fold {fold+1} Training on {len(inner_train_subset)}, Validating on {len(inner_val_subset)}, Testing on {len(outer_test_subset)}.")

    best_model_path = f"{SAVED_MODELS_PATH}/best_model_fold_{fold+1}.pth"
    best_val_acc_fold = 0.0
    epochs_without_improvement = 0

    for epoch in range(1, EPOCHS + 1):
        epoch_start_time = time.time()
        
        print(f"\n--- Fold {fold+1}, Epoch {epoch}/{EPOCHS} ---")
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, DEVICE)
        val_loss, val_acc = validate(model, val_loader, criterion, DEVICE)

        scheduler.step()

        epoch_duration = time.time() - epoch_start_time

        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f}")
        print(f"  Epoch Duration: {epoch_duration:.2f} seconds")
        
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('Accuracy/val', val_acc, epoch)
        writer.add_scalar('Performance/seconds_per_epoch', epoch_duration, epoch)
        writer.add_scalar('Learning_Rate', optimizer.param_groups[0]['lr'], epoch)

        if val_acc > best_val_acc_fold:
            best_val_acc_fold = val_acc
            epochs_without_improvement = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"  -> New best inner validation accuracy: {best_val_acc_fold:.4f}. Model state saved.")
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= PATIENCE:
                print(f"  Early stopping at epoch {epoch}.")
                break
    
    print(f"\n--- Fold {fold+1} Final Evaluation ---")
    print(f"Loading best model for fold {fold+1} (achieved {best_val_acc_fold:.4f} on inner val set).")
    # Load the best model weights identified during the inner loop
    model.load_state_dict(torch.load(best_model_path))
    
    # Evaluate this best model on the completely unseen outer test set
    _, final_fold_test_acc = validate(model, test_loader, criterion, DEVICE)
    print(f"Performance of best model on the OUTER TEST SET for fold {fold+1}: {final_fold_test_acc:.4f}")

    fold_test_results.append(final_fold_test_acc)
    writer.add_scalar('Accuracy/test', final_fold_test_acc)
    
    hparams = { 'learning_rate': LEARNING_RATE, 'batch_size': BATCH_SIZE, **model_params }
    final_metrics = {
        'hparam/test_accuracy': final_fold_test_acc,
        'hparam/best_inner_val_accuracy': best_val_acc_fold
    }
    writer.add_hparams(hparams, final_metrics)
    writer.close()

# ==============================================================================
# 4. AGGREGATE AND REPORT CV RESULTS
# ==============================================================================
mean_test_acc = np.mean(fold_test_results)
std_test_acc = np.std(fold_test_results)

print(f"\n{'='*30} FINAL UNBIASED PERFORMANCE SUMMARY {'='*30}")
print(f"Unbiased test accuracies from each fold: {[f'{acc:.4f}' for acc in fold_test_results]}")
print(f"Average Test Accuracy from CV: {mean_test_acc:.4f}")
print(f"Standard Deviation of Test Accuracy from CV: {std_test_acc:.4f}")
print(f"-> Estimated Generalization Performance: {mean_test_acc:.4f} \u00B1 {std_test_acc:.4f}")

print("\nCross-validation finished.")
print("To view TensorBoard logs, run the following command in your terminal:")
print(f"tensorboard --logdir runs/{MODEL_FOLDER_NAME}")

Total data for cross-validation: 500 samples.

============================== OUTER FOLD 1/3 ==============================
Loading pretrained weights from: ../RADIO/saved_models/RADIO1_MSRGNN/deployable_model.pth

--- Transfer Learning Report ---
Successfully loaded 138 / 140 layers.

Skipped 1 layers:
  - reasoner.panel_pos_emb.weight (Shape Mismatch. Pretrained: torch.Size([9, 128]), Model: torch.Size([4, 128]))
--------------------------------

Fold 1 Model has 4,804,547 trainable parameters.
Fold 1 Training on 283, Validating on 50, Testing on 167.

--- Fold 1, Epoch 1/100 ---


Training: 100%|██████████| 9/9 [00:01<00:00,  7.84it/s]


Epoch Loss: 1.3821, Accuracy: 0.4594


Validating: 100%|██████████| 2/2 [00:00<00:00,  4.22it/s]


  Train Loss: 1.3821 | Train Acc: 0.4594
  Val Loss:   1.6095 | Val Acc:   0.2000
  Epoch Duration: 1.63 seconds
  -> New best inner validation accuracy: 0.2000. Model state saved.

--- Fold 1, Epoch 2/100 ---


...

Epoch Loss: 0.0289, Accuracy: 0.9929


Validating: 100%|██████████| 2/2 [00:00<00:00,  4.15it/s]


  Train Loss: 0.0289 | Train Acc: 0.9929
  Val Loss:   3.3996 | Val Acc:   0.4314
  Epoch Duration: 1.53 seconds
  Early stopping at epoch 23.

--- Fold 3 Final Evaluation ---
Loading best model for fold 3 (achieved 0.5098 on inner val set).


Validating: 100%|██████████| 6/6 [00:00<00:00,  9.81it/s]

Performance of best model on the OUTER TEST SET for fold 3: 0.4458

============================== FINAL UNBIASED PERFORMANCE SUMMARY ==============================
Unbiased test accuracies from each fold: ['0.5090', '0.4132', '0.4458']
Average Test Accuracy from CV: 0.4560
Standard Deviation of Test Accuracy from CV: 0.0398
-> Estimated Generalization Performance: 0.4560 ± 0.0398

Cross-validation finished.
To view TensorBoard logs, run the following command in your terminal:
tensorboard --logdir runs/O3_5-MSRGNN-TRANSFER_RADIO


# RADIO-2 to O3 TL

In [32]:
from sklearn.model_selection import train_test_split
VAL_SPLIT_SIZE = 0.15

MODEL_FOLDER_NAME = "O3_5-MSRGNN-TRANSFER_RADIO_2"

# ==============================================================================
# 2. DATASET & FOLD SETUP
# ==============================================================================
print(f"Total data for cross-validation: {len(full_dataset)} samples.")

kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
fold_test_results = []

# ==============================================================================
# 3. CROSS-VALIDATION LOOP
# ==============================================================================
for fold, (train_outer_idx, test_outer_idx) in enumerate(kfold.split(np.zeros(len(full_dataset)))):
    print(f"\n{'='*30} OUTER FOLD {fold+1}/{N_SPLITS} {'='*30}")

    SAVED_MODELS_PATH = f"./saved_models/{MODEL_FOLDER_NAME}/"

    if not os.path.exists(SAVED_MODELS_PATH):
        os.makedirs(SAVED_MODELS_PATH)
    writer = SummaryWriter(log_dir=f'runs/{MODEL_FOLDER_NAME}/fold_{fold+1}')

    inner_train_idx, inner_val_idx = train_test_split(
        train_outer_idx,
        test_size=VAL_SPLIT_SIZE,
        shuffle=True, # Shuffle before splitting
        random_state=42
    )

    inner_train_subset = Subset(full_dataset, inner_train_idx)
    inner_val_subset = Subset(full_dataset, inner_val_idx)
    outer_test_subset = Subset(full_dataset, test_outer_idx)

    train_loader = DataLoader(inner_train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(inner_val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(outer_test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    model = OddOneOutMSRGNN(**model_params).to(DEVICE)

    load_partial_state_dict(
        model,
        "../RADIO/saved_models/RADIO2_MSRGNN/deployable_model.pth"
    )

    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
    scaler = None

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Fold {fold+1} Model has {total_params:,} trainable parameters.")
    print(f"Fold {fold+1} Training on {len(inner_train_subset)}, Validating on {len(inner_val_subset)}, Testing on {len(outer_test_subset)}.")

    best_model_path = f"{SAVED_MODELS_PATH}/best_model_fold_{fold+1}.pth"
    best_val_acc_fold = 0.0
    epochs_without_improvement = 0

    # --- Inner Training Loop for the Fold ---
    for epoch in range(1, EPOCHS + 1):
        epoch_start_time = time.time()
        
        print(f"\n--- Fold {fold+1}, Epoch {epoch}/{EPOCHS} ---")
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, DEVICE)
        val_loss, val_acc = validate(model, val_loader, criterion, DEVICE)

        scheduler.step()

        epoch_duration = time.time() - epoch_start_time

        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f}")
        print(f"  Epoch Duration: {epoch_duration:.2f} seconds")
        
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('Accuracy/val', val_acc, epoch)
        writer.add_scalar('Performance/seconds_per_epoch', epoch_duration, epoch)
        writer.add_scalar('Learning_Rate', optimizer.param_groups[0]['lr'], epoch)

        if val_acc > best_val_acc_fold:
            best_val_acc_fold = val_acc
            epochs_without_improvement = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"  -> New best inner validation accuracy: {best_val_acc_fold:.4f}. Model state saved.")
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= PATIENCE:
                print(f"  Early stopping at epoch {epoch}.")
                break
    
    print(f"\n--- Fold {fold+1} Final Evaluation ---")
    print(f"Loading best model for fold {fold+1} (achieved {best_val_acc_fold:.4f} on inner val set).")
    # Load the best model weights identified during the inner loop
    model.load_state_dict(torch.load(best_model_path))
    
    # Evaluate this best model on the completely unseen outer test set
    _, final_fold_test_acc = validate(model, test_loader, criterion, DEVICE)
    print(f"Performance of best model on the OUTER TEST SET for fold {fold+1}: {final_fold_test_acc:.4f}")

    fold_test_results.append(final_fold_test_acc)
    writer.add_scalar('Accuracy/test', final_fold_test_acc)
    
    hparams = { 'learning_rate': LEARNING_RATE, 'batch_size': BATCH_SIZE, **model_params }
    final_metrics = {
        'hparam/test_accuracy': final_fold_test_acc,
        'hparam/best_inner_val_accuracy': best_val_acc_fold
    }
    writer.add_hparams(hparams, final_metrics)
    writer.close()

# ==============================================================================
# 4. AGGREGATE AND REPORT CV RESULTS
# ==============================================================================
mean_test_acc = np.mean(fold_test_results)
std_test_acc = np.std(fold_test_results)

print(f"\n{'='*30} FINAL UNBIASED PERFORMANCE SUMMARY {'='*30}")
print(f"Unbiased test accuracies from each fold: {[f'{acc:.4f}' for acc in fold_test_results]}")
print(f"Average Test Accuracy from CV: {mean_test_acc:.4f}")
print(f"Standard Deviation of Test Accuracy from CV: {std_test_acc:.4f}")
print(f"-> Estimated Generalization Performance: {mean_test_acc:.4f} \u00B1 {std_test_acc:.4f}")

print("\nCross-validation finished.")
print("To view TensorBoard logs, run the following command in your terminal:")
print(f"tensorboard --logdir runs/{MODEL_FOLDER_NAME}")

Total data for cross-validation: 500 samples.

============================== OUTER FOLD 1/3 ==============================
Loading pretrained weights from: ../RADIO/saved_models/RADIO2_MSRGNN/deployable_model.pth

--- Transfer Learning Report ---
Successfully loaded 138 / 140 layers.

Skipped 1 layers:
  - reasoner.panel_pos_emb.weight (Shape Mismatch. Pretrained: torch.Size([9, 128]), Model: torch.Size([4, 128]))
--------------------------------

Fold 1 Model has 4,804,547 trainable parameters.
Fold 1 Training on 283, Validating on 50, Testing on 167.

--- Fold 1, Epoch 1/100 ---


Training: 100%|██████████| 9/9 [00:01<00:00,  8.26it/s]


Epoch Loss: 1.3835, Accuracy: 0.4452


Validating: 100%|██████████| 2/2 [00:00<00:00,  4.38it/s]


  Train Loss: 1.3835 | Train Acc: 0.4452
  Val Loss:   1.6083 | Val Acc:   0.2400
  Epoch Duration: 1.55 seconds
  -> New best inner validation accuracy: 0.2400. Model state saved.

--- Fold 1, Epoch 2/100 ---


...

Epoch Loss: 0.0036, Accuracy: 1.0000


Validating: 100%|██████████| 2/2 [00:00<00:00,  4.16it/s]


  Train Loss: 0.0036 | Train Acc: 1.0000
  Val Loss:   3.2471 | Val Acc:   0.4706
  Epoch Duration: 1.54 seconds
  Early stopping at epoch 20.

--- Fold 3 Final Evaluation ---
Loading best model for fold 3 (achieved 0.5294 on inner val set).


Validating: 100%|██████████| 6/6 [00:00<00:00,  9.55it/s]

Performance of best model on the OUTER TEST SET for fold 3: 0.5060

============================== FINAL UNBIASED PERFORMANCE SUMMARY ==============================
Unbiased test accuracies from each fold: ['0.4491', '0.4431', '0.5060']
Average Test Accuracy from CV: 0.4661
Standard Deviation of Test Accuracy from CV: 0.0284
-> Estimated Generalization Performance: 0.4661 ± 0.0284

Cross-validation finished.
To view TensorBoard logs, run the following command in your terminal:
tensorboard --logdir runs/O3_5-MSRGNN-TRANSFER_RADIO_2
